# WireScope — Transmission Line Reflections

**Goal:** Understand how impedance mismatches create signal reflections on a cable and why Ethernet and similar networks must terminate cables correctly.

By the end, you will:
1. Compute the **reflection coefficient** \( \Gamma = \frac{Z_L - Z_0}{Z_L + Z_0} \)
2. Simulate a pulse reflecting on open, shorted, and matched loads.
3. Relate reflection timing to **cable length** and **signal velocity**.
4. Explain how these reflections cause bit errors and define the **collision domain** in networks.

In [23]:
# Common imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
from ipywidgets import interact, FloatSlider

# Networking/packet analysis
import pyshark
import scapy.all as scapy
import networkx as nx

# Helper: inline plots
%matplotlib inline


# WireScope — Transmission Line Reflections

**Goal:** Visualize how signal reflections occur when a digital pulse meets an impedance mismatch on a transmission line.

This lab connects signal physics with networking hardware (Ethernet, coax, twisted-pair, etc.).
You’ll simulate:
1. A pulse traveling down a cable.
2. How reflections arise when the far-end impedance differs.
3. How termination and cable length affect signal quality.

We'll relate these to **Ethernet PHY** requirements — why 50Ω coax and twisted-pair impedance matching matters.


In [24]:
# Define Physical Model

# Cable parameters
v = 2e8 # propagation velocity
Z0 = 50 # characteristic impedance (ohms)
length = 100 # cable length in meters
dt = 1e-9 # time step (1 ns)
t = np.arange(0, 2*length/v, dt) # simulate one round-trip

# Source and load impedance
Zs = 50 # source impedance (ohms)
ZL_open = 1e6 # open circuit
ZL_match = 50 # matched load
ZL_short = 1e-6 # short circuit

# Compute Reflection Coefficient
def reflection_coefficient(ZL, Z0):
	return (ZL - Z0) / (ZL + Z0)

Gamma_open = reflection_coefficient(ZL_open, Z0)
Gamma_match = reflection_coefficient(ZL_match, Z0)
Gamma_short = reflection_coefficient(ZL_short, Z0)

print(f"Open: {Gamma_open:.2f}, Match: {Gamma_match:.2f}, Short: {Gamma_short:.2f}")

Open: 1.00, Match: 0.00, Short: -1.00


In [ ]:
# Simulate a pulse traveling down the line
# We emit a finite-width pulse and model its far-end reflection arriving after 2*length/v.
# The reflection window must be u(t - delay) - u(t - (delay + width)).
# We also extend plotting to highlight the reflection time.
def simulate_reflection(ZL, Z0, t, length, v, pulse_width=50e-9):
    Gamma = reflection_coefficient(ZL, Z0)
    # launch pulse at t=0
    pulse = np.heaviside(t, 1) - np.heaviside(t - pulse_width, 1)

    # reflection occurs after round-trip time (2 * length / v)
    delay = 2 * length / v
    reflected_window = (np.heaviside(t - delay, 1) - np.heaviside(t - (delay + pulse_width), 1))
    reflected = Gamma * reflected_window

    signal_total = pulse + reflected
    return signal_total, delay, pulse_width

# Extend time to go slightly beyond the reflection
T_end = 2*length/v + 200e-9
_t = np.arange(0, T_end, dt)

plt.figure(figsize=(10,4))
for ZL, label in [(ZL_open,"Open"), (ZL_match,"Matched"), (ZL_short,"Short")]:
    s, delay, pw = simulate_reflection(ZL, Z0, _t, length, v)
    plt.plot(_t*1e9, s, label=label)
# Annotate expected reflection arrival
plt.axvline(x=delay*1e9, color='k', linestyle='--', alpha=0.5, label='Reflection ETA')
plt.xlabel("Time [ns]"); plt.ylabel("Voltage (arb.)")
plt.title("Reflected Pulse for Different Terminations (reflection marked)")
plt.legend(); plt.grid(True)
plt.xlim(0, (delay + 3*pw)*1e9)
plt.show()


### 🔍 Prediction

Before running the next cell:

- If the cable is **open**, what sign do you expect for the reflection (positive or inverted)?
- If it's **shorted**, how will the reflection look?
- If it's **matched**, will you see a reflection at all?

Write your predictions here:


(Students write under this cell — small engagement step.)

# Quick check: verify a visible reflection for Open/Short vs Matched
s_open, delay, pw = simulate_reflection(ZL_open, Z0, _t, length, v)
s_match, _, _ = simulate_reflection(ZL_match, Z0, _t, length, v)
pre = (_t >= (delay - 5*pw)) & (_t < delay)
post = (_t >= delay) & (_t < delay + 5*pw)
contrast_open = float(np.max(s_open[post]) - np.mean(s_open[pre]))
contrast_match = float(np.max(s_match[post]) - np.mean(s_match[pre]))
print(f"Reflection contrast (Open) ~ {contrast_open:.3f}, (Matched) ~ {contrast_match:.3f}")

# Learning Note:
# The second bump is the reflection.
# Open = same polarity, Short = inverted, Matched = no bump.

When a digital pulse (like an Ethernet bit) reaches the cable end:
- If the end impedance ≠ cable impedance, some energy **reflects back**.
- These reflections interfere with upcoming bits, causing jitter or misreads.
- Ethernet solves this by **impedance matching** and **differential signaling**.

If cable length ≈ bit time × velocity, reflections overlap with live data — this defines **collision domain length**.

> Note on fix: The reflection bump looked flat earlier because the window for the reflected pulse was mis-specified and the plot ended right at the round-trip time. We corrected the window to u(t - delay) - u(t - (delay + width)), increased the pulse width to 50 ns for visibility, and extended the time axis slightly beyond the reflection arrival. You should now clearly see the second bump at the dashed line for open/short, and no bump for the matched load.


In [ ]:
# 🧩 Interactive Reflection Visualizer (optional)
from ipywidgets import interact, FloatSlider

def reflection_demo(length=50, v=2e8, ZL=1e6):
    Z0 = 50
    t = np.arange(0, 3e-6, 1e-9)
    Gamma = (ZL - Z0) / (ZL + Z0)
    pulse_width = 20e-9
    pulse = np.where((t >= 0) & (t <= pulse_width), 1.0, 0.0)
    delay = 2 * length / v
    reflected = np.where((t >= delay) & (t <= delay + pulse_width), Gamma, 0.0)
    total = pulse + reflected

    plt.figure(figsize=(8,4))
    plt.plot(t*1e9, total)
    plt.title(f"Reflection Simulation — Length={length:.0f} m, Γ={Gamma:.2f}")
    plt.xlabel("Time [ns]")
    plt.ylabel("Voltage (arb.)")
    plt.grid(True)
    plt.show()

interact(reflection_demo,
         length=FloatSlider(10, 10, 200, step=10, description="Cable (m)"),
         v=FloatSlider(2e8, 1.5e8, 3e8, step=0.1e8, description="Velocity"),
         ZL=FloatSlider(0, 1000, 1e6, step=50, description="Load Ω (∞=open)"));


### 🌐 Networking Analogy — Collisions and Echoes

| Concept | Physical meaning | Networking meaning |
|----------|------------------|--------------------|
| Reflection coefficient Γ | Fraction of signal returning | Portion of bit energy echoing back |
| Matched termination | No reflection | Stable symbol sampling |
| Open/short cable | Total reflection | Signal bounce, garbled bits |
| Round-trip delay | 2 × length / velocity | Collision detection window |
| TDR pulse test | Reflected echo reveals fault | Used in cable testers (Time-Domain Reflectometers) |

In Ethernet, a TDR uses the same math to locate cable breaks — by measuring the time until the echo returns and dividing by \(2v\).

| Concept | Physical Behavior | Networking Consequence |
|----------|------------------|------------------------|
| Impedance mismatch | Reflected voltage wave | Bit error or collision |
| Proper termination | Full energy absorbed | Clean transitions, stable PHY |
| Cable length limit | Round-trip delay < slot time | CSMA/CD can detect collisions |
| Reflection coefficient | Signal echo strength | Return loss in dB |
| Short/Open fault | +1 or -1 reflection | Detected by TDR (Time-Domain Reflectometer) |

So “wire reflections” are the **physical layer origin** of why Ethernet once had maximum cable lengths (e.g., 100m for twisted pair).


In [29]:
sig_open = simulate_reflection(ZL_open, Z0, t, length, v)
f = np.fft.fftreq(len(t), dt)
S_open = np.abs(np.fft.fft(sig_open))

plt.figure(figsize=(10,4))
plt.plot(f[:len(f)//2]/1e6, S_open[:len(S_open)//2])
plt.xlabel("Frequency [MHz]"); plt.ylabel("|Spectrum|")
plt.title("Frequency Response with Open-Termination Reflection")
plt.grid(True); plt.show()

AttributeError: module 'numpy.fft' has no attribute 'fftreq'

### 🧠 How to Read the Plot

- The **first bump** at \(t=0\) ns is the transmitted pulse.
- The **second** bump, appearing after \(t = 2L/v\), is the reflected pulse.
- **Open load:** same polarity reflection (Γ ≈ +1)
- **Short load:** inverted reflection (Γ ≈ –1)
- **Matched load:** no visible reflection (Γ ≈ 0)

**Why this matters:**  
Reflections distort voltage levels. If they overlap the next bit period, the receiver may interpret a “1” as “0” or vice versa.
Ethernet’s maximum cable length ensures the round-trip delay is shorter than one collision-detection slot (51.2 µs for 10 Mb/s).


### ✅ Quick Self-Check

1. What does a reflection coefficient of +1, 0, and −1 correspond to physically?
2. Why does matching \(Z_L = Z_0\) eliminate reflections?
3. If a 100 m Cat-5 cable shows an echo 1 µs after the pulse, what’s the approximate propagation speed?
4. How does Ethernet’s 100 m limit relate to round-trip delay?

*(Answer these before checking the reference sheet.)*


### 🛠️ Real-World Connection

If you have access to a network cable tester (Fluke, Klein, etc.), it performs exactly this simulation in hardware:
it sends a voltage pulse, measures reflections, and displays **distance to fault** based on the echo delay.

So your simulation = digital version of a **Time-Domain Reflectometer (TDR)** — the same instrument testers use to find opens/shorts.
